# OECD - Semantic Role Labeling

** Best to run this notebook on a GPU for a reasonable execution time

This notebook generates (Subject, Verb, Object) tuples for the entire OECD corpus of documents. 

### 1. Import the data

In [ ]:
from gensim.test.utils import datapath
from gensim import utils
from nltk.tokenize import word_tokenize, sent_tokenize
import re
import json
import string 
import nltk
from nltk.corpus import stopwords
nltk.download('punkt')

corpus_path = datapath('../data-files/processed_ngram_ner_data.json')
with open(corpus_path, encoding='utf-8') as f:
    datajson = json.load(f)

corpus = ''
for key in datajson:
    corpus += datajson[key] + ' '
    
f = open('../data-files/ngram_replacements.json')
ngram_replacements = json.load(f)
        
def replace_all(text, dic):
    for i, j in dic.items():
        text = text.replace(i, j)
    return text

def get_preprocessed_corpus(corpus):
    global ngram_replacements
    
    # split corpus into sentences
    sentences = sent_tokenize(corpus)
    cleaned_sentences = []
    for sentence in sentences:
        # replace ngrams with single tokens
        cleaned_sentence = replace_all(sentence, ngram_replacements)        
        cleaned_sentence = cleaned_sentence.replace('(', '').replace(')', '')
        cleaned_sentences.append(cleaned_sentence)
            
    return cleaned_sentences

In [ ]:
# processed corpus into cleaned list of sentences
processed_sentences = get_preprocessed_corpus(corpus)

If the `processed_sentences` are stored to file (data-files/srl_corpus.pkl), you can also load it with the code in the cell below:

In [ ]:
# import pickle
# with open('/content/drive/MyDrive/srl-allennlp/data-files/srl_corpus.pkl', 'rb') as f:
#     processed_sentences = pickle.load(f)

### 2. Initialise or prepare SRL model

In [ ]:
# initialise the bilstm model for SRL using the allennlp wrapper
from allennlp_models import pretrained
predictor = pretrained.load_predictor('structured-prediction-srl') # Bilstm model. use 'structured-prediction-srl-bert' as an alternative

If you encounter any runtime / installation errors with allennlp, try installing spacy-transformers below:

In [ ]:
# !pip install git+https://github.com/explosion/spacy-transformers

### 3. Load the OECD named entities dataset

In [ ]:
import pandas as pd

df = pd.read_csv('../data-files/full-flair-ner-list-oecd-with-single-tokens.csv')
orgs_df = df[df['entity_type'] == 'ORG'] 
orgs_df.head()

## 4. Generate SRL tuples for each sentence in the corpus

In [ ]:
# function to parse SRL output per sentence
def get_srl_tag_words(sentence):
    tokens = re.findall(r'\[(.*?)\]', sentence)
    verb = None
    arg0 = None 
    arg1 = None
    for token in tokens:
        if  token.startswith('V:'):
            verb = token.replace('V:','').strip()
        if  token.startswith('ARG0:'):
            arg0 = token.replace('ARG0:','').strip()
        if  token.startswith('ARG1:'):
            arg1 = token.replace('ARG1:','').strip()

    return verb, arg0, arg1
            
preds_list = []
index = 1
for sent in processed_sentences:
    print('sentence', index, '/', str(len(processed_sentences)))# 
    if (len(sent.split()) < 512):
        preds = predictor.predict(sent)
        for i in range(0, len(preds["verbs"])):
            verb, arg0, arg1 = get_srl_tag_words(preds["verbs"][i]['description'])
            if (((verb is not None) and (arg0 is not None) and (arg1 is not None)) and (len(verb) > 0 and len(arg0) > 0 and len(arg1) > 0)):
                preds_list.append(preds)
    index += 1
                

### 5. Save RAW SRL results to file

In [ ]:
import pickle
with open('../data-files/srl_predictions_big.pkl', 'wb') as f:
    pickle.dump(preds_list, f)

In [ ]:
# with open('/content/drive/MyDrive/srl-allennlp/data-files/srl_predictions.pkl', 'rb') as f:
#     srl_results = pickle.load(f)

In [ ]:
# print("number of pairs: ", len(srl_results))